Visualization of the results of the model
=================================


A GAMCR model has two main methods to produce results:

* `.predict_streamflow(matJ)`: to get the hydrograph

* `.predict_transfer_function(X)`: to get the transfer functions (RRDs)

# 1. Simple visualization

Below we provide is simple code to show the observed and predicted hydrographs, and the predicted average NRF transfer function.

In [ ]:
import GAMCR

site= 'Basel_notflashy'
# You should change this line specifying the path of your model
model.load_model(os.path.join(root,'{0}/{0}_best_model.pkl'.format(site)))
X, matJ, y, timeyear, dates = model.load_data(os.path.join(root, '{0}/data/'.format(site)), max_files=20)

yhat = model.predict_streamflow(matJ)
plt.figure()
plt.plot(yhat, label='estimation')
plt.plot(y, label='ground truth', linestyle='--')
plt.plot([],[], color='white', label='NSE: {0}'.format(GAMCR.nse(y, yhat)))
plt.title(site)
plt.legend()
plt.show()

a = np.mean(model.gam._modelmat(X), axis=0)
smooth_P = np.kron( GAMCR.build_custom_matrix(model.L), np.dot(a.reshape(-1,1), a.reshape(1,-1)) )

coeffs = model.gam.get_coeffs().reshape(-1)
print( coeffs.T @ smooth_P @ coeffs)

plt.figure()
H = model.predict_transfer_function(X)
plt.figure()
plt.plot(np.array([i/24 for i in range(24*3)]), H.mean(axis=0)[:24*3], label='estimation')
plt.title(site)
plt.legend()
plt.show()

# 2. Advanced analysis of the results

By launching the code below, you can compute different information on the learned transfer functions, such as:
- the global average NRD/RDD
- the average NRF/RRD over some ensembles (you can stratify either by precipitation intensity, antecendent wetness or by both)
- the area, mean, peak and peak lag of the transfer function over different ensembles for the precipitation intensity

If ERRA data is available, the ensembles will be automatically set to the ones used by ERRA to allow a fair comparison. Otherwise, we can define yourself the ensemble.

Once the results have been computed, they are saved in a pickle file. The notebook: 

In [ ]:
import pandas as pd
import numpy as np
import sys
from joblib import Parallel, delayed
import GAMCR
save_folder= '/mydata/watres/quentin/code/FLOW/hourly_analysis/RES_GAMCR/simulated_data/'
import os
import pickle
from get_data_from_ERRA import *


ls_stations = ['Lugano', 'Pully', 'Basel']

ls_modes = ['flashy', 'notflashy']

for station in ls_stations:
    for mode in ls_modes:
        site = '{0}_{1}'.format(station, mode)
        model = GAMCR.model.GAMCR(lam=0.1)
        datapath = os.path.join(save_folder, site, '{0}_best_model.pkl'.format(site))
        model.load_model(datapath)
        site_folder = os.path.join(save_folder, site)

        try:
            dicERRA = get_data_from_ERRA()
            nblocks = len(dicERRA[site]['groups_precip'])
            groups_precip = dicERRA[site]['groups_precip']
            groups_precip[0] = (1,groups_precip[0][1])
            model.compute_statistics(site_folder, site, nblocks = nblocks, min_precip = groups_precip[0][0], groups_precip=groups_precip, max_files=96)
        except:
            model.compute_statistics(site_folder, site)

![Screen shot of NRF averaged over different ranges of precipitation intensity.](../../_static/nb_visualization.png)